In [1]:
import os
import cv2
import numpy as np
import torch
import clip
from PIL import Image
import pandas as pd

In [2]:
# Load CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# Paths to data
template_path = "./LicensePlateDataTest/Character_templates"
plate_path = "./LicensePlateDataTest/License_plate"


In [3]:
# Load character templates
templates = []
labels = []
for file in os.listdir(template_path):
    if file.endswith(".jpg"):
        img_path = os.path.join(template_path, file)
        template_img = cv2.imread(img_path)
        _, binary_img = cv2.threshold(template_img, 128, 255, cv2.THRESH_BINARY_INV)
        # cv2.imshow("binary_img", binary_img)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()
        # break
        img = preprocess(Image.fromarray(binary_img)).unsqueeze(0).to(device)
        templates.append(img)
        labels.append(os.path.splitext(file)[0])  # Use file name as label

templates = torch.cat(templates, 0)

# Encode character templates with CLIP
with torch.no_grad():
    template_features = model.encode_image(templates)


In [4]:
# show img templates[0]
# img0 = np.transpose(np.array(templates[0]), (1, 2, 0))
# cv2.imshow('img', img0)
# img0.shape
# cv2.waitKey(0)
# cv2.destroyAllWindows()
# binary_img.shape

def show_tensor_img(tensor_img):
    img_numpy = np.transpose(np.array(tensor_img), (1, 2, 0))
    cv2.imshow('img', img_numpy)
    print(img_numpy.shape)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [5]:
show_tensor_img(templates[0])

(224, 224, 3)


In [6]:
# template_features.shape
templates.shape
# labels

torch.Size([65, 3, 224, 224])

In [5]:
# def extract_characters(image, num_chars=7):
#     """Divide the image into character regions."""
#     h, w = image.shape
#     char_width = w // num_chars
#     return [image[:, i * char_width : (i + 1) * char_width] for i in range(num_chars)]


def extract_characters(image, num_chars=7):
    """Divide the image into character regions."""
    h, w = image.shape
    x_range_list = [
        (10, 65),
        (70, 125),
        (145, 200),
        (200, 255),
        (255, 310),
        (315, 370),
        (370, 425),
    ]
    y_range = (20, 120)
    # char_width = w // num_chars
    # return [image[:, i * char_width : (i + 1) * char_width] for i in range(num_chars)]
    characters_images = []
    for x_range in x_range_list:
        x_min, x_max = x_range
        characters_images.append(image[y_range[0] : y_range[1], x_min:x_max])

    return characters_images

In [1]:
plate_path = os.path.join(plate_path, "ahy1N6L77_blue_False.jpg")
plate_img = cv2.imread(plate_path, cv2.IMREAD_GRAYSCALE)
_, binary_img = cv2.threshold(plate_img, 128, 255, cv2.THRESH_BINARY_INV)

# Extract characters
char_regions = extract_characters(binary_img)

plate_text = ""
for char_img in char_regions:

    # Resize to match template size
    char_img = cv2.resize(char_img, (templates.shape[-1], templates.shape[-2]))
    # cv2.imshow("Char", char_img)
    # while True:
    #     key = cv2.waitKey(0)
    #     if key == ord("q"):
    #         exit()
    #     elif key == ord(" "):
    #         break
    print(char_img.shape)   
    char_tensor = preprocess(Image.fromarray(char_img)).unsqueeze(0).to(device)
    show_tensor_img(char_tensor)
    # Encode the character
    with torch.no_grad():
        char_features = model.encode_image(char_tensor)

    # Compare to template features
    similarity = (char_features @ template_features.T).squeeze(0)
    best_match = similarity.argmax().item()

    # Append recognized character
    plate_text += labels[best_match]

NameError: name 'os' is not defined

In [ ]:


def recognize_plate(plate_path):
    """Recognize license plate characters."""
    # Read and preprocess the plate image
    plate_img = cv2.imread(plate_path, cv2.IMREAD_GRAYSCALE)
    _, binary_img = cv2.threshold(plate_img, 128, 255, cv2.THRESH_BINARY_INV)

    # Extract characters
    char_regions = extract_characters(binary_img)

    plate_text = ""
    for char_img in char_regions:

        # Resize to match template size
        char_img = cv2.resize(char_img, (templates.shape[-1], templates.shape[-2]))
        cv2.imshow("Char", char_img)
        while True:
            key = cv2.waitKey(0)
            if key == ord("q"):
                exit()
            elif key == ord(" "):
                break

        char_tensor = preprocess(Image.fromarray(char_img)).unsqueeze(0).to(device)

        # Encode the character
        with torch.no_grad():
            char_features = model.encode_image(char_tensor)

        # Compare to template features
        similarity = (char_features @ template_features.T).squeeze(0)
        best_match = similarity.argmax().item()

        # Append recognized character
        plate_text += labels[best_match]

    return plate_text

In [ ]:
# Process all plates and save results
results = []
for file in os.listdir(plate_path):
    if file.endswith(".jpg"):
        full_path = os.path.join(plate_path, file)
        plate_number = recognize_plate(full_path)
        print(f"Plate number for {file} is {plate_number}")
        results.append({"Filename": file, "LicensePlate": plate_number})

# Save results to CSV
# df = pd.DataFrame(results)
# df.to_csv("results.csv", index=False)
# print("Recognition complete. Results saved to 'results.csv'.")


Plate number for ahy4SD8V2_blue_False.jpg is hnq4JJJJJ
Plate number for ahy1N6L77_blue_False.jpg is hnqJNJJJJ
Plate number for ahy5LBR4E_blue_False.jpg is hnqJJJJ4J


: 